#CafChem Teaching - Featurizing and SciKit-learn.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChemTeach/blob/main/notebooks/Featurizing_SKLearn_CafChem.ipynb)

## This notebook allows you to:
- Featurize molecules based on SMILES strings.
- Use a Random Forest regressor model to fit a common solubility dataset.

## Requirements:
- If using on Colab, t will install all needed libraries.
- Can use your local environment or a CPU runtime on Colab.

## Setup
- Installing rdkit only needed is using Colab.

In [1]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 46.5 MB/s eta 0:00:00


In [15]:
import numpy as np
import pandas as pd
import time
import pickle as pkl
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, Descriptors
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

## Functions
The functions below allow you to:
- Featurize a list of molecules.
- Check the features for NaN and remove those entries.
- Fit your model to the data.
- Predict solubility of a new molecule

In [3]:
def rdkit_featurize(smiles_list: list, target_list: list, print_flag: bool):
  '''
    Takes a list of SMILES strings and creates features using the RDKit set of
    descriptors.

      Args:
        smiles_list: List of SMILES to featurize
        target_list: list of the ground trutch values for each molecule
        print_flag: True: print the number of descriptors for each molecule
                    FAalse: do not print anything
      Returns:
        X: 2D list of features (rows are molecules, columns are features)
        y: list of target values
        mols: list of RDKit mol objects
        legend: list of SMILES strings (should be identical to input list,
                unless a molecule could not be featurized, in which case that molecule
                is left out)
  '''
  X = []
  mols = []
  legend = []
  y = []
  add_flag = True
  for i,smile in enumerate(smiles_list):
    try:
      mol = Chem.MolFromSmiles(smile)
      dictionary_descriptors = Chem.Descriptors.CalcMolDescriptors(mol)
      temp_vec = []
      for key in dictionary_descriptors:
        temp_vec.append(dictionary_descriptors[key])
        add_flag = True
      X.append(temp_vec)
      mols.append(mol)
      legend.append(smile)
      y.append(target_list[i])
      if print_flag:
        print(f"{len(temp_vec)} descriptors calculated for: {smile}")
        print("--------------------------------------------------------")
    except:
      print(f"Could not featurize molecule {i}")

  print(f"Total number of molecules: {len(X)}")
  print(f"Total number of descriptors per molecule: {len(X[0])}")

  return X, y, mols, legend

def check_for_nan(f: list, y: list, Xa: list):
  '''
    Accepts the features array, the targets list, and the SMILES list. Checks
    the feature array for NaN. Removes any rows with NaN from features, targets
    and SMILES.

      Args:
        f: 2D list of features (rows are molecules, columns are features)
        y: list of target values
        Xa: list of SMILES strings
      Returns:
        f: trimmed 2D list of features (rows are molecules, columns are features)
        y: trimmed list of target values
        Xa: trimmed list of SMILES strings
  '''
  f = np.array(f)
  y = np.array(y)
  Xa = np.array(Xa)
  nan_indicies = np.isnan(f)
  bad_rows = []
  for i, row in enumerate(nan_indicies):
      for item in row:
          if item == True:
              if i not in bad_rows:
                  #print(f"Row {i} has a NaN.")
                  bad_rows.append(i)

  print(f"Old dimensions are: {f.shape}.")

  for j,i in enumerate(bad_rows):
      k=i-j
      f = np.delete(f,k,axis=0)
      y = np.delete(y,k,axis=0)
      Xa = np.delete(Xa,k,axis=0)
      #print(f"Deleting row {k} from arrays.")

  print(f"New dimensions are: {f.shape}")

  return f, y, Xa

def random_forest_regressor(X_train, y_train, X_test, y_test):
  '''
    Fits the SKLearn random firest regressor to the input training data. Calculates
    the R2 score for the training and test data.

      Args:
        X_train: 2D list of training features (rows are molecules, columns are features)
        y_train: list of training target values
        X_test: 2D list of validation features (rows are molecules, columns are features)
        y_test: list of validation target values

      Returns:
        rf: trained random forest regressor
        train_r2: R2 score for the training data
        test_r2: R2 score for the test data
  '''
  rf = RandomForestRegressor(n_estimators=100, random_state=42)
  rf.fit(X_train, y_train)

  train_pred = rf.predict(X_train)
  test_pred = rf.predict(X_test)

  train_r2 = r2_score(y_train, train_pred)
  test_r2 = r2_score(y_test, test_pred)

  return rf, train_r2, test_r2

def predict_with_model(model, smiles_to_predict, scaler = None):
  '''
    Accepts a single SMILES string and featurizes it. Then applies scaling if appropriate,
    and predicts the target value with the model

      Args:
        model: the trained model to use.
        smiles_to_predict: the test molecule.
        scaler (optional): the scaler to apply.
      Returns:
        prediction: the predicted value.
  '''
  mol = Chem.MolFromSmiles(smiles_to_predict)
  dictionary_descriptors = Chem.Descriptors.CalcMolDescriptors(mol)
  temp_vec = []
  for key in dictionary_descriptors:
    temp_vec.append(dictionary_descriptors[key])

  if scaler is None:
    X_scaled = [temp_vec]
  else:
    X_scaled = scaler.transform([temp_vec])

  prediction = model.predict(X_scaled)

  print(f"The predicted value for {smiles_to_predict} is: {prediction[0]}")
  return prediction

## Read in data
- Upload the delaney.csv file from the CafChemTeach repository.

In [4]:
df = pd.read_csv('/content/delaney.csv')
df.head()

,Compound ID,measured log(solubility:mol/L),ESOL predicted log(solubility:mol/L),SMILES
0,"1,1,1,2-Tetrachloroethane",-2.18,-2.794,ClCC(Cl)(Cl)Cl
1,"1,1,1-Trichloroethane",-2.00,-2.232,CC(Cl)(Cl)Cl
2,"1,1,2,2-Tetrachloroethane",-1.74,-2.549,ClC(Cl)C(Cl)Cl
3,"1,1,2-Trichloroethane",-1.48,-1.961,ClCC(Cl)Cl
4,"1,1,2-Trichlorotrifluoroethane",-3.04,-3.077,FC(F)(Cl)C(F)(Cl)Cl


## Featurize the molecules
- Set the name of the SMILES and target columns
- check for NaNs
- Scale the features
- Split the data into training and testing sets

In [5]:
X, y, mols, smiles = rdkit_featurize(df['SMILES'], df['measured log(solubility:mol/L)'], print_flag=False)

Total number of molecules: 1144
Total number of descriptors per molecule: 217


In [6]:
X, y, smiles = check_for_nan(X, y, smiles)

Old dimensions are: (1144, 217).
New dimensions are: (1144, 217)


In [7]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Train the model
- Print out the R2 values

In [9]:
model, train_r2, test_r2 = random_forest_regressor(X_train, y_train, X_test, y_test)

In [10]:
print(f"Train R2: {train_r2}")
print(f"Test R2: {test_r2}")

Train R2: 0.9858122988986158
Test R2: 0.9098266320591084


## Predict the target value for a novel molecule

In [18]:
pred = predict_with_model(model, 'c1ccc(F)cc1', scaler)

The predicted value for c1ccc(F)cc1 is: -1.8126000000000015


## Save and/or Load your model

In [13]:
model_filename= "my_model.pkl"

In [16]:
with open(model_filename, "wb") as f:
    pkl.dump(model, f)
print(f"Model saved to {model_filename}")

Model saved to my_model.pkl


In [17]:
with open(model_filename, "rb") as f:
    model = pkl.load(f)
print(f"Model loaded from {model_filename}")

Model loaded from my_model.pkl
